In [1]:
import gym
import numpy as np
import tensorflow as tf

In [2]:
D = 4
H = 10
batch_size = 5
learning_rate = 1e-2
gamma = 0.99

In [3]:
tf.reset_default_graph()
observations = tf.placeholder(tf.float32,[None,D],name='input_x')
w1 = tf.Variable(np.random.normal(size=[D,H]),dtype=tf.float32,name='w1')
b1 = tf.Variable(np.zeros([H,]),dtype=tf.float32,name='b1')
layer1 = tf.nn.relu(tf.matmul(observations,w1) + b1)
w2 = tf.Variable(np.random.normal(size=[H,1]),dtype=tf.float32,name='w2')
b2 = tf.Variable(np.zeros([1,]),dtype=tf.float32,name='b2')
score = tf.matmul(layer1,w2) + b2
probability = tf.nn.sigmoid(score)

tvars = tf.trainable_variables()
input_y = tf.placeholder(tf.float32,[None,1],name='input_y')
advantages = tf.placeholder(tf.float32,name='reward')

loglik = -tf.log(input_y*(input_y-probability) + (1-input_y)*(input_y +probability))
cost = tf.reduce_mean(loglik * advantages)
newGrads =  tf.gradients(cost,tvars)

adam = tf.train.AdamOptimizer(learning_rate=learning_rate)
W1Grad = tf.placeholder(tf.float32,name="batch_grad1")
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
batcbGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batcbGrad,tvars))


In [4]:
def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [ ]:
env = gym.make('CartPole-v0')
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000
init = tf.initialize_all_variables()

with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset()
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        if reward_sum/batch_size > 100 or rendering == True : 
            rendering = True
            
        x = np.reshape(observation,[1,D])
        
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        xs.append(x)
        y = 1 if action == 0 else 0
        ys.append(y)

        observation, reward, done, info = env.step(action)
        reward_sum += reward

        drs.append(reward) 

        if done: 
            episode_number += 1
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            tfp = tfps
            xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]

            discounted_epr = discount_rewards(epr)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            if episode_number % batch_size == 0: 
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print 'Average reward for episode %f.  Total average reward %f.' % (reward_sum/batch_size, running_reward/batch_size)
                
                if reward_sum/batch_size > 800: 
                    print "Task solved in",episode_number,'episodes!'
                    x = [env.reset()]
                    while True:
                        env.render()
                        prob = sess.run(probability,feed_dict={observations : x})
                        if prob > np.random.uniform():
                            observation,reward,done,_ = env.step(1)
                            x = [ observation ]
                            if done:
                                x = [env.reset()]
                        else:
                            observation,reward,done,_ = env.step(0)
                            x = [ observation ]
                            if done:
                                x = [env.reset()]
                    env.close()
                    break
                reward_sum = 0
            
            observation = env.reset()
        
print episode_number,'Episodes completed.'


INFO:gym.envs.registration:Making new env: CartPole-v0
[2017-02-16 18:16:11,876] Making new env: CartPole-v0


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.
[2017-02-16 18:16:11,889] From <ipython-input-10-260f85658fba>:7 in <module>.: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Average reward for episode 15.400000.  Total average reward 15.400000.
Average reward for episode 23.200000.  Total average reward 15.478000.
Average reward for episode 25.800000.  Total average reward 15.581220.
Average reward for episode 30.800000.  Total average reward 15.733408.
Average reward for episode 38.800000.  Total average reward 15.964074.
Average reward for episode 20.000000.  Total average reward 16.004433.
Average reward for episode 22.600000.  Total average reward 16.070389.
Average reward for episode 25.200000.  Total average reward 16.161685.
Average reward for episode 30.600000.  Total average reward 16.306068.
Average reward for episode 23.200000.  Total average reward 16.375007.
Average reward for episode 32.800000.  Total average reward 16.539257.
Average reward for episode 25.800000.  Total average reward 16.631865.
Average reward for episode 28.800000.  Total average reward 16.753546.
Average reward for episode 21.600000.  Total average reward 16.802010.
Averag